# Calculating dissolved volatile concentrations

The `calculate_dissolved_volatiles()` function calcutions the concentration of dissolved H$_2$O and CO$_2$ in the liquid at a given pressure-temperature condition and with a given H$_2$O-CO$_2$ fluid composition, defined as the mole fraction of H$_2$O in an H$_2$O-CO$_2$ fluid (XH$_2$O$^{fluid}$). The default MagmaSat model relies on the underlying functionatlity of MELTS, whose basic function is to calculate the equilibrium phase assemblage given the bulk composition of the system and pressure-temperature conditions. To calculate dissolved volatile concentrations thus requires computing the equilibrium state of a system at fixed pressure and temperature over a range of bulk volatile concentrations until a solution is found that satisfies the user defined fluid composition.

First, the function makes an initial guess at the appropriate bulk volatile concentrations by finding the minimum dissolved volatile concentrations in the liquid at saturation, while asserting that the weight fraction of H$_2$O/$\Sigma$volatiles in the system is equal to the user input mole fraction of H$_2$O/$\Sigma$volatiles in the fluid. This is done by increasing the H$_2$O and CO$_2$ concentrations appropriately until a fluid phase is stable. Once fluid saturation is determined, the code then performs directional, iterative, and progressively more refined searches, increasing the proportion of H$_2$O or CO$_2$ in the system if the mole fraction of H$_2$O calculated in the fluid is greater than or less than that defined by the user, respectively. Four iterative searches are performed; the precision of the match between the calculated and defined XH$_2$O$^{fluid}$ increases from 0.1 in the first iteration to 0.01, 0.001, and finally to 0.0001. Thus, the calculated dissolved volatile concentrations correspond to a system with XH$_2$O$^{fluid}$ within 0.0001 of the user defined value.

**Method structure:**<br>
> Single sample: `def calculate_dissolved_volatiles(self, sample, temperature, pressure, X_fluid=1, verbose=False).result`

>ExcelFile batch process: `def calculate_dissolved_volatiles(self, temperature, pressure, X_fluid=1, print_status=False)`

**Required inputs:**<br>
>`sample`: *Only for single-sample calculations.* The composition of a sample. A single sample may be passed as a dictionary of values, with compositions of oxides in wt%.

>`temperature`, `pressure`, and `X_fluid`: the temperature in $^{\circ}$C, the pressure in bars, and the mole fraction of H$_2$O in the H$_2$O-CO$_2$ fluid, XH$_2$O$^{fluid}$. Temperature and pressure of the sample or samples must be passed unless an ExcelFile object with a column for temperature and/or pressure is passed to `sample`. XH$_2$O$^{fluid}$ is optional, with a default value of 1 (pure H$_2$O fluid). If a numerical (float) value is passed for either temperature, pressure, or X_fluid, that will be the value used for one or all samples. If, alternatively, the user wishes to use temperature, pressure, and/or X_fluid information in their ExcelFile object, the title of the column containing temperature, pressure, or X_fluid data should be passed in quotes (as a string) to `temperature`,  `pressure`, and/or `X_fluid`, respectively. Note for batch calculations that if temperature, pressure, or XH$_2$O$^{fluid}$ information exists in the ExcelFile but a single numerical value is defined for one or both of these variables, both the original information plus the values used for the calculations will be returned.

**Optional inputs:**<br>
>`verbose`: *Only for single-sample calculations.* Default value is False. If set to True, additional parameters are returned in a dictionary: H$_2$O and CO$_2$ concentrations in the fluid in mole fraction, temperature, pressure, and proportion of the fluid in the system in wt%.

> `print_status`: *Only for ExcelFile batch calcualtions.* The default value is False. If True is passed, the progress of the calculation will be printed to the terminal. The user may desire to see the status of the calculation, as this particular function can be quite slow, averaging between 3-5 seconds per sample. #TODO test this more and verify average compute time.

**Calculated outputs:**<br>
>If a single sample is passed to `sample`, a dictionary with keys 'H2O' and 'CO2' corresponding to the calculated dissolved H$_2$O and CO$_2$ concentrations in the liquid is returned (plus additional variables 'temperature' in $^{\circ}$C, 'pressure' in bars, 'XH2O_fl', 'XCO2_fl', and 'FluidProportion_wtper' (the proportion of the fluid in the system in wt%) if `verbose` is set to True). 

>If mutliple samples are passed as an ExcelFile object, a pandas DataFrame is returned with sample information plus calculated dissolved H$_2$O and CO$_2$ concentrations in the liquid, the fluid composition in mole fraction, and the proportion of the fluid in the system in wt%. Pressure (in bars) and Temperature (in $^{\circ}$C) columns are always returned.

In [1]:
import sys
sys.path.insert(0, '../')

import VESIcal as v

## For an entire dataset
### Import an Excel file

In [2]:
myfile = v.ExcelFile('../manuscript/example_data.xlsx')

### Do the calculation

In [3]:
dissolved = myfile.calculate_dissolved_volatiles(temperature=900.0, pressure=1000.0, X_fluid=0.5, print_status=True)
dissolved

Calculating sample BT-ex
Calculating sample TVZMa-ex
Calculating sample TVZOh-ex
Calculating sample Oh48-FTIR1-MI1-a
Calculating sample Oh48-FTIR1-MI1-b
Calculating sample Oh48-FTIR1-MI1-IRc
Calculating sample Oh50-4.1
Calculating sample Oh50-4.2
Calculating sample Oh49-4.1
Calculating sample Oh49-4.2
Calculating sample Ma55-5a.1
Calculating sample Ma57-3b.2
Calculating sample Ma57-3c.1
Calculating sample Ma57-3c.2
Done!


,SiO2,TiO2,Al2O3,Fe2O3,Cr2O3,FeO,MnO,MgO,NiO,CoO,...,Temp,H2O_liq_VESIcal,CO2_liq_VESIcal,XH2O_fl_VESIcal,XCO2_fl_VESIcal,FluidProportion_wt_VESIcal,Temperature_C_VESIcal,Pressure_bars_VESIcal,X_fluid_input_VESIcal,Model
Label,,,,,,,,,,,,,,,,,,,,,
BT-ex,77.50,0.0800,12.50,0.207,0,0.4730,0.0000,0.0300,0,0,...,900,2.383793,0.032469,0.499693,0.500307,1.293601,900.0,1000.0,0.5,MagmaSat
TVZMa-ex,78.37,0.1300,11.94,0.000,0,0.9900,0.0400,0.0500,0,0,...,800,2.401527,0.034647,0.499329,0.500671,1.291067,900.0,1000.0,0.5,MagmaSat
TVZOh-ex,77.90,0.0800,12.15,0.000,0,0.9500,0.0500,0.0600,0,0,...,900,2.407038,0.033450,0.499777,0.500223,1.294307,900.0,1000.0,0.5,MagmaSat
Oh48-FTIR1-MI1-a,78.27,0.0298,12.02,0.000,0,0.9828,0.0336,0.0515,0,0,...,950,2.410223,0.033708,0.499415,0.500585,1.292148,900.0,1000.0,0.5,MagmaSat
Oh48-FTIR1-MI1-b,78.27,0.0298,12.02,0.000,0,0.9828,0.0336,0.0515,0,0,...,1025,2.410223,0.033708,0.499415,0.500585,1.292148,900.0,1000.0,0.5,MagmaSat
Oh48-FTIR1-MI1-IRc,78.27,0.0298,12.02,0.000,0,0.9828,0.0336,0.0515,0,0,...,925,2.410223,0.033708,0.499415,0.500585,1.292148,900.0,1000.0,0.5,MagmaSat
Oh50-4.1,77.91,0.0984,12.07,0.000,0,1.0556,0.0257,0.0999,0,0,...,862,2.404744,0.033622,0.499928,0.500072,1.293136,900.0,1000.0,0.5,MagmaSat
Oh50-4.2,77.91,0.0984,12.07,0.000,0,1.0556,0.0257,0.0999,0,0,...,770,2.404744,0.033622,0.499928,0.500072,1.293136,900.0,1000.0,0.5,MagmaSat
Oh49-4.1,77.92,0.0099,12.11,0.000,0,1.0020,0.0672,0.0546,0,0,...,855,2.409442,0.033409,0.499807,0.500193,1.293091,900.0,1000.0,0.5,MagmaSat


## For a single sample

### Extract a single sample from your dataset

In [4]:
SampleName = 'BT-ex'
extracted_bulk_comp = myfile.get_sample_oxide_comp(SampleName)

### Do the calculation

In [5]:
v.calculate_dissolved_volatiles(sample=extracted_bulk_comp, temperature=900.0, pressure=2000.0, X_fluid=0.5).result

{'CO2': 0.0704089917125897, 'H2O': 3.40549411877139}

In [7]:
myfile.save_excelfile('dissolved.xlsx', calculations=[dissolved])

Saved dissolved.xlsx
